In [3]:
setwd('/import/DB/zhang//data')

In [1]:
library(infercnv)

The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)



In [2]:
library(Seurat)

Warning message:
"package 'Seurat' was built under R version 4.2.3"
Attaching SeuratObject



In [10]:
sce1 = readRDS('OS.final.rds')
epiMat=as.data.frame(GetAssayData( sce1 , slot='counts',assay='RNA'))

Warning message in asMethod(object):
"sparse->dense coercion: allocating vector of size 3.3 GiB"


In [6]:
Tcell = readRDS('./T.rds')

In [13]:
T_cellsMat=as.data.frame(GetAssayData( Tcell, slot='counts',assay='RNA'))

Warning message in asMethod(object):
"sparse->dense coercion: allocating vector of size 3.8 GiB"


In [16]:
T_cellsMat=T_cellsMat[,sample(1:ncol(T_cellsMat),3000)]

In [17]:
head(T_cellsMat,2)

,AGGTGTTGTCCTTTGC-2_1,ATGATCGCATCGAACT-4_1,AATTCCTAGCGGGTAT-2_1,GACTCTCTCTCGACCT-1_1,ATTACTCAGTAGTGCG-3_1,TGTTACTTCGTTAGTG-1_6,TGGGATTGTATCCTTT-1_10,GCTGGGTTCTTCCACG-1_6,AAGTGAAGTGAATGTA-2_1,GGTTCTCGTCTGTGAT-4_1,...,TACTTGTCATTAAAGG-3_1,ACTGCAACATTCGGGC-1_6,CAGAGCCGTATTCTCT-2_1,TCAGTTTGTGCACAAG-2_1,CTCCATGAGAGGTTTA-1_3,GTTGAACTCGACACCG-2_1,CTTCCTTGTGAACCGA-1_1,GGAGAACCACTGAGTT-4_1,GTTACCCCAAACTAGA-2_1,TTTCCTCCAAATAAGC-3_1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AL627309.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AL669831.5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
epiMat = epiMat[rownames(T_cellsMat),]

In [20]:

dat=cbind(epiMat,T_cellsMat)

In [21]:


groupinfo=data.frame(v1=colnames(dat),
                     v2=c(rep('epi',ncol(epiMat)),
                          rep('spike-T_cells',1500),
                          rep('ref-T_cells',1500)))

In [22]:
library(AnnoProbe)
geneInfor=annoGene(rownames(dat),"SYMBOL",'human')
colnames(geneInfor)
geneInfor=geneInfor[with(geneInfor, order(chr, start)),c(1,4:6)]
geneInfor=geneInfor[!duplicated(geneInfor[,1]),]
length(unique(geneInfor[,1]))
head(geneInfor)
## 这里可以去除性染色体
# 也可以把染色体排序方式改变
dat=dat[rownames(dat) %in% geneInfor[,1],]
dat=dat[match( geneInfor[,1], rownames(dat) ),] 
dim(dat)
expFile='expFile.txt'
write.table(dat,file = expFile,sep = '\t',quote = F)
groupFiles='groupFiles.txt'
head(groupinfo)
write.table(groupinfo,file = groupFiles,sep = '\t',quote = F,col.names = F,row.names = F)
head(geneInfor)
geneFile='geneFile.txt'
write.table(geneInfor,file = geneFile,sep = '\t',quote = F,col.names = F,row.names = F)

Setting options('download.file.method.GEOquery'='auto')

Setting options('GEOquery.inmemory.gpl'=FALSE)

AnnoProbe v 0.1.0  welcome to use AnnoProbe!
If you use AnnoProbe in published research, please acknowledgements:
We thank Dr.Jianming Zeng(University of Macau), and all the members of his bioinformatics team, biotrainee, for generously sharing their experience and codes.

Warning message in annoGene(rownames(dat), "SYMBOL", "human"):
"9.26% of input IDs are fail to annotate... "


[1] "SYMBOL"   "biotypes" "ENSEMBL"  "chr"      "start"    "end"

[1] 18040

,SYMBOL,chr,start,end
,<chr>,<chr>,<int>,<int>
10,AL627309.1,chr1,89295,133723
48,LINC01409,chr1,778747,810065
51,FAM87B,chr1,817371,819837
52,LINC01128,chr1,825138,859446
53,LINC00115,chr1,826206,827522
55,FAM41C,chr1,868071,876903


[1] 18040 22544

,v1,v2
,<chr>,<chr>
1,AAACGCTCAAGCAATA-1_1,epi
2,AAAGGGCTCCATCAGA-1_1,epi
3,AAAGTCCCACGGCACT-1_1,epi
4,AACACACCATGCCGAC-1_1,epi
5,AACCACAGTCTTTCAT-1_1,epi
6,AACCCAACAGAAGCGT-1_1,epi


,SYMBOL,chr,start,end
,<chr>,<chr>,<int>,<int>
10,AL627309.1,chr1,89295,133723
48,LINC01409,chr1,778747,810065
51,FAM87B,chr1,817371,819837
52,LINC01128,chr1,825138,859446
53,LINC00115,chr1,826206,827522
55,FAM41C,chr1,868071,876903


In [23]:
library(Seurat)
library(gplots)
library(ggplot2)

expFile='expFile.txt' 
groupFiles='groupFiles.txt'  
geneFile='geneFile.txt'

library(infercnv)
infercnv_obj = CreateInfercnvObject(raw_counts_matrix=expFile,
                                    annotations_file=groupFiles,
                                    delim="\t",
                                    gene_order_file= geneFile,
                                    ref_group_names=c('ref-T_cells'))


Attaching package: 'gplots'


The following object is masked from 'package:stats':

    lowess


Warning message:
"package 'ggplot2' was built under R version 4.2.3"


INFO [2024-07-27 10:32:58] Parsing matrix: expFile.txt
INFO [2024-07-27 10:34:51] Parsing gene order file: geneFile.txt
INFO [2024-07-27 10:34:51] Parsing cell annotations file: groupFiles.txt
INFO [2024-07-27 10:34:52] ::order_reduce:Start.
INFO [2024-07-27 10:34:52] .order_reduce(): expr and order match.
INFO [2024-07-27 10:34:53] ::process_data:order_reduce:Reduction from positional data, new dimensions (r,c) = 18040,22544 Total=65474651.8941963 Min=0 Max=8.66706721321292.
INFO [2024-07-27 10:34:56] num genes removed taking into account provided gene ordering list: 603 = 3.34257206208426% removed.
INFO [2024-07-27 10:34:57] -filtering out cells < 100 or > Inf, removing 0 % of cells
WARN [2024-07-27 10:34:58] Please use "options(scipen = 100)" before running infercnv if you are using the analysis_mode="subclusters" option or you may encounter an error while the hclust is being generated.
INFO [2024-07-27 10:35:10] validating infercnv_obj


In [ ]:
infercnv_obj2 = infercnv::run(infercnv_obj,
                              cutoff=0.1, # cutoff=1 works well for Smart-seq2, and cutoff=0.1 works well for 10x Genomics
                              out_dir= "infercnv_output",  # dir is auto-created for storing outputs
                              cluster_by_groups=F,   # cluster
                              analysis_mode = "samples",
                              hclust_method="ward.D2", plot_steps=F)

In [33]:
packageVersion('infercnv')

[1] '1.14.2'